<a href="https://colab.research.google.com/github/inventor-2-developer/homes_sales/blob/main/Home_Sales_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [80.6 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu b

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# Read in the dataset from the AWS S3 bucket.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/nflx-data-science-adv/week-5/Assignment/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True)
df.show()

+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|_c0|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|  0|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|  1|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|  2|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|  3|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|  4|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|  5|5aa00529-0533-46b...|2019-01-30|      2017|

In [10]:
# 1. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [54]:
# 2. What is the average price for a four bedroom house sold in 2019 rounded to two decimal places?
sql="""SELECT price FROM home_sales   """
spark.sql(sql).show()

#spark.sql("select * from DATA where rlike(date,'^[0-9]*$')").show()

+------+
| price|
+------+
|936923|
|379628|
|417866|
|239895|
|424418|
|218712|
|419199|
|323956|
|181925|
|258710|
|167864|
|337527|
|229896|
|210247|
|398667|
|437958|
|437375|
|288650|
|308313|
|177541|
+------+
only showing top 20 rows



In [37]:
# 3. What is the average price of a home for each year that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
sql=""" SELECT date, bedrooms,bathrooms FROM home_sales WHERE bedrooms ==3 and bathrooms==3"""
spark.sql(sql).show()


+----------+--------+---------+
|      date|bedrooms|bathrooms|
+----------+--------+---------+
|2020-06-16|       3|        3|
|2021-08-06|       3|        3|
|2019-02-27|       3|        3|
|2020-05-09|       3|        3|
|2021-05-31|       3|        3|
|2019-10-04|       3|        3|
|2019-07-15|       3|        3|
|2022-06-05|       3|        3|
|2019-04-17|       3|        3|
|2019-10-28|       3|        3|
|2020-02-12|       3|        3|
|2021-11-18|       3|        3|
|2022-03-06|       3|        3|
|2021-06-18|       3|        3|
|2019-02-06|       3|        3|
|2019-03-31|       3|        3|
|2022-04-01|       3|        3|
|2022-06-12|       3|        3|
|2019-09-17|       3|        3|
|2020-05-11|       3|        3|
+----------+--------+---------+
only showing top 20 rows



In [38]:
# 4. What is the average price of a home for each year that have 3 bedrooms, 3 bathrooms, and are less than 2,000 square feet rounded to two decimal places?
sql = """ SELECT bedrooms, bathrooms,date_built, sqft_living FROM home_sales WHERE bedrooms==3 and bathrooms==3 and sqft_living >2000

"""
spark.sql(sql).show()


+--------+---------+----------+-----------+
|bedrooms|bathrooms|date_built|sqft_living|
+--------+---------+----------+-----------+
|       3|        3|      2016|       2137|
|       3|        3|      2011|       2471|
|       3|        3|      2015|       2197|
|       3|        3|      2011|       2356|
|       3|        3|      2016|       2130|
|       3|        3|      2016|       2025|
|       3|        3|      2014|       2271|
|       3|        3|      2017|       2162|
|       3|        3|      2010|       2049|
|       3|        3|      2015|       2356|
|       3|        3|      2011|       2371|
|       3|        3|      2010|       2383|
|       3|        3|      2017|       2226|
|       3|        3|      2015|       2485|
|       3|        3|      2015|       2051|
|       3|        3|      2015|       2487|
|       3|        3|      2015|       2234|
|       3|        3|      2010|       2325|
|       3|        3|      2014|       2309|
|       3|        3|      2013| 

In [41]:
# 5. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are less than $350,000?
# Although this is a small dataset, determine the run time for this query.
sql = """ SELECT price, view FROM home_sales WHERE price >350000

"""
spark.sql(sql).show()



+------+----+
| price|view|
+------+----+
|936923|  76|
|379628|  23|
|417866|   0|
|424418|   4|
|419199|   6|
|398667|   7|
|437958|  26|
|437375|  34|
|377485|  23|
|417215|  48|
|372151|  16|
|391574|  10|
|419543|  24|
|843191|  91|
|429183|  49|
|395178|  37|
|429030|  23|
|396508|  25|
|399016|  23|
|409011|   1|
+------+----+
only showing top 20 rows



In [ ]:
# 6. Cache the the temporary table home_sales.


DataFrame[]

In [ ]:
# 7. Check if the table is cached.


True

In [ ]:
# 8. Using the cached data, run the query that filters out the view ratings with average price of less than $350,000. 
# Determine the runtime and compare it to uncached runtime.



sql = """

"""
spark.sql(sql).show()




+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.8723046779632568 seconds ---


In [ ]:
# 9. Partition the home sales dataset by the date_built field.


In [ ]:
# 10.Read the formatted parquet data.


In [ ]:
# 11. Create a temporary table for the parquet data.


In [ ]:
# 12. Run the query that filters out the view ratings with average price of less than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 



sql = """

"""
spark.sql(sql).show()



+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.8514270782470703 seconds ---


In [ ]:
# 13. Uncache the home_sales temporary table.


DataFrame[]

In [ ]:
# 14. Check if the home_sales is no longer cached


all clear
